# 📊 EDA Regulatório MEC/SERES — **Deltas (2018 vs 2019+)** + Série Histórica (2007–2025)

Este notebook analisa dois períodos (até 2018 e desde 2019) e **mostra a variação (Δ)** entre eles — isto é, **quanto aumentou ou diminuiu** em **pontos percentuais (pp)** ou em **dias**, conforme o indicador.

✅ **Importante:** não é um “comparativo lado a lado” tradicional; os gráficos principais mostram o **Δ (2019+ − 2018)**.

---

## Arquivos de entrada
- `total_2018_CINE.xlsx`  → **df_2018**
- `total_2019_CINE.xlsx`  → **df_2019plus**

> Se seus nomes estiverem diferentes, ajuste na célula de leitura.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)


## 1) Leitura (df_2018 e df_2019plus)

In [ ]:
ARQ_2018 = "total_2018_CINE.xlsx"
ARQ_2019 = "total_2019_CINE.xlsx"

df_2018 = pd.read_excel(ARQ_2018)
df_2019plus = pd.read_excel(ARQ_2019)

print("df_2018:", df_2018.shape)
print("df_2019plus:", df_2019plus.shape)

df_2018.head()


## 2) Funções utilitárias (padronização mínima + gráficos Δ com rótulo)

In [ ]:
def normalize_missing(s: pd.Series) -> pd.Series:
    x = s.copy()
    x = x.astype(str).str.strip()
    x = x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})
    return x

def bar_delta(labels, values, title, xlabel, ylabel, horizontal=True, figsize=(10,5)):
    """Bar de DELTA (pode ter valores negativos). Remove eixo numérico e coloca rótulo."""
    values = np.asarray(values, dtype=float)
    labels = [str(l) for l in labels]

    if len(values) == 0:
        print("⚠️ Sem dados para plotar.")
        return

    colors = ["#1f77b4" if v >= 0 else "#7f7f7f" for v in values]

    plt.figure(figsize=figsize)
    if horizontal:
        bars = plt.barh(labels, values, color=colors)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)

        ax = plt.gca()
        ax.spines["bottom"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.xaxis.set_ticks([])

        for b in bars:
            w = b.get_width()
            txt = f"{w:.1f}".replace(".", ",")
            plt.text(w, b.get_y() + b.get_height()/2, txt,
                     va="center", ha="left" if w>=0 else "right", fontsize=9)
        plt.axvline(0, linewidth=1)
        plt.gca().invert_yaxis()
    else:
        bars = plt.bar(labels, values, color=colors)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)

        ax = plt.gca()
        ax.spines["left"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.yaxis.set_ticks([])

        for b in bars:
            h = b.get_height()
            txt = f"{h:.1f}".replace(".", ",")
            plt.text(b.get_x() + b.get_width()/2, h, txt, va="bottom", ha="center", fontsize=9)
        plt.axhline(0, linewidth=1)

    plt.tight_layout()
    plt.show()

def preparar_base(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    if "NO_DO_PROCESSO" in out.columns:
        out = out.drop(columns=["NO_DO_PROCESSO"])

    # CINE Geral (preencher vazios)
    if "AREA_GERAL_CINE" in out.columns:
        out["AREA_GERAL_CINE"] = normalize_missing(out["AREA_GERAL_CINE"]).fillna("Não informado")
    else:
        out["AREA_GERAL_CINE"] = "Não informado"

    # UF
    if "UF_PROCESSO" in out.columns:
        out["UF"] = normalize_missing(out["UF_PROCESSO"])
    elif "UF_CADASTRO" in out.columns:
        out["UF"] = normalize_missing(out["UF_CADASTRO"])
    else:
        out["UF"] = np.nan

    # Ano
    if "ANO_DO_PROTOCOLO" in out.columns:
        out["ANO_DO_PROTOCOLO"] = pd.to_numeric(out["ANO_DO_PROTOCOLO"], errors="coerce")

    # Modalidade_norm (alias)
    if "MODALIDADE_NORM" in out.columns:
        out["Modalidade_norm"] = normalize_missing(out["MODALIDADE_NORM"]).str.upper()
    elif "MODALIDADE" in out.columns:
        out["Modalidade_norm"] = normalize_missing(out["MODALIDADE"]).str.upper()
    else:
        out["Modalidade_norm"] = np.nan

    # Pública/Privada
    if "CATEGORIA_ADMINISTRATIVA" in out.columns:
        cat = normalize_missing(out["CATEGORIA_ADMINISTRATIVA"]).str.upper()
        out["PublicaPrivada"] = np.where(cat.str.contains("PÚBLIC|PUBLIC", na=False), "PÚBLICA", "PRIVADA")
    else:
        out["PublicaPrivada"] = np.nan

    # Âmbito (Sistema de ensino)
    if "SISTEMA_DE_ENSINO" in out.columns:
        sist = normalize_missing(out["SISTEMA_DE_ENSINO"]).str.upper()
        out["AmbitoAdministrativo"] = np.select(
            [sist.str.contains("FEDERAL", na=False),
             sist.str.contains("ESTADUAL", na=False),
             sist.str.contains("MUNICIPAL", na=False)],
            ["FEDERAL","ESTADUAL","MUNICIPAL"],
            default="OUTROS"
        )
    else:
        out["AmbitoAdministrativo"] = "DESCONHECIDO"

    # Endereço divergente flag
    if "ENDERECO_DIVERGENTE" in out.columns:
        s = out["ENDERECO_DIVERGENTE"].astype(str).str.strip().str.upper()
        out["endereco_divergente_flag"] = np.select(
            [s.isin(["SIM","TRUE","1","VERDADEIRO","T"]),
             s.isin(["NÃO","NAO","FALSE","0","FALSO","F"])],
            [1,0],
            default=np.nan
        )
    else:
        out["endereco_divergente_flag"] = np.nan

    # Sede EAD flag (0/1)
    raw = out["IS_SEDE_EAD"] if "IS_SEDE_EAD" in out.columns else out.get("is_sede_ead", np.nan)
    raw_str = pd.Series(raw, index=out.index).astype(str).str.strip().str.upper()
    out["is_sede_ead_flag"] = np.select(
        [raw_str.isin(["SIM","TRUE","1","VERDADEIRO","T","S"]),
         raw_str.isin(["NÃO","NAO","FALSE","0","FALSO","F","N"])],
        [1,0],
        default=pd.to_numeric(raw, errors="coerce")
    )
    out.loc[~out["is_sede_ead_flag"].isin([0,1]), "is_sede_ead_flag"] = np.nan

    # Datas e tempo (preferência: DATA_DE_ENTRADA_FASE_ATUAL, senão DATA_DO_ULTIMO_ATO)
    if "DATA" in out.columns:
        out["DATA"] = pd.to_datetime(out["DATA"], errors="coerce")
    fim_col = None
    if "DATA_DE_ENTRADA_FASE_ATUAL" in out.columns:
        out["DATA_DE_ENTRADA_FASE_ATUAL"] = pd.to_datetime(out["DATA_DE_ENTRADA_FASE_ATUAL"], errors="coerce")
        fim_col = "DATA_DE_ENTRADA_FASE_ATUAL"
    elif "DATA_DO_ULTIMO_ATO" in out.columns:
        out["DATA_DO_ULTIMO_ATO"] = pd.to_datetime(out["DATA_DO_ULTIMO_ATO"], errors="coerce")
        fim_col = "DATA_DO_ULTIMO_ATO"

    if ("DATA" in out.columns) and (fim_col is not None):
        out["TEMPO_TRAMITACAO_DIAS"] = (out[fim_col] - out["DATA"]).dt.days
        out.loc[out["TEMPO_TRAMITACAO_DIAS"] < 0, "TEMPO_TRAMITACAO_DIAS"] = np.nan
    else:
        out["TEMPO_TRAMITACAO_DIAS"] = np.nan

    return out

def pct_series(df, col, drop_na=True):
    s = df[col]
    if drop_na:
        s = s.dropna()
    return (s.value_counts(normalize=True) * 100).sort_values(ascending=False)

def delta_pct(pct_2019, pct_2018):
    idx = pct_2019.index.union(pct_2018.index)
    a = pct_2019.reindex(idx).fillna(0)
    b = pct_2018.reindex(idx).fillna(0)
    return a - b


## 3) Preparar bases + base unificada (só para série histórica)

In [ ]:
df_2018 = preparar_base(df_2018)
df_2019plus = preparar_base(df_2019plus)

df_all = pd.concat([df_2018, df_2019plus], ignore_index=True)

display(df_2018.head(2))
display(df_2019plus.head(2))


## 4) Série histórica: processos por Ano do Protocolo (2007–2025)

In [ ]:
proc_por_ano = df_all.groupby("ANO_DO_PROTOCOLO").size().sort_index()
proc_por_ano = proc_por_ano[proc_por_ano.index.notna()]

labels = proc_por_ano.index.astype(int).astype(str)
values = proc_por_ano.values

plt.figure(figsize=(11,5))
bars = plt.bar(labels, values)
plt.title("Número de processos por Ano do Protocolo (2007–2025)")
plt.xlabel("Ano do Protocolo")
plt.ylabel("Quantidade")

ax = plt.gca()
ax.spines["left"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.yaxis.set_ticks([])

for b in bars:
    h = b.get_height()
    plt.text(b.get_x()+b.get_width()/2, h, f"{int(h):,}".replace(",","."), ha="center", va="bottom", fontsize=8)

plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


## 5) Δ (pp): Sede EAD (Sim/Não) — quanto mudou

In [ ]:
pct_sede_2018 = (df_2018["is_sede_ead_flag"].dropna().value_counts(normalize=True) * 100)
pct_sede_2019 = (df_2019plus["is_sede_ead_flag"].dropna().value_counts(normalize=True) * 100)

map_lbl = {0: "Não", 1: "Sim"}
pct_sede_2018.index = pct_sede_2018.index.map(map_lbl)
pct_sede_2019.index = pct_sede_2019.index.map(map_lbl)

delta = delta_pct(pct_sede_2019, pct_sede_2018).sort_values(ascending=False)

display(pd.DataFrame({"% 2018": pct_sede_2018, "% 2019+": pct_sede_2019, "Δ (pp)": delta}).fillna(0))

bar_delta(
    labels=delta.index,
    values=delta.values,
    title="Δ (pp) — Sede EAD (2019+ − 2018)",
    xlabel="Pontos percentuais (pp)",
    ylabel="Sede EAD",
    horizontal=True,
    figsize=(7,3)
)


## 6) Δ (pp): Modalidade (distribuição geral) — Top mudanças

In [ ]:
pct_mod_2018 = pct_series(df_2018, "Modalidade_norm")
pct_mod_2019 = pct_series(df_2019plus, "Modalidade_norm")

delta_mod = delta_pct(pct_mod_2019, pct_mod_2018).sort_values(ascending=False)

tbl = pd.DataFrame({"% 2018": pct_mod_2018, "% 2019+": pct_mod_2019, "Δ (pp)": delta_mod}).fillna(0)
display(tbl.sort_values("Δ (pp)", ascending=False).head(15))

top = tbl.sort_values("Δ (pp)", ascending=False).head(15)["Δ (pp)"]
bar_delta(
    labels=top.index,
    values=top.values,
    title="Top 15 Δ (pp) — Modalidade (2019+ − 2018)",
    xlabel="Δ em pontos percentuais (pp)",
    ylabel="Modalidade",
    horizontal=True,
    figsize=(10,6)
)


## 7) Δ (pp): Pública vs Privada e Âmbito administrativo

In [ ]:
pct_pp_2018 = pct_series(df_2018, "PublicaPrivada")
pct_pp_2019 = pct_series(df_2019plus, "PublicaPrivada")
delta_pp = delta_pct(pct_pp_2019, pct_pp_2018)

display(pd.DataFrame({"% 2018": pct_pp_2018, "% 2019+": pct_pp_2019, "Δ (pp)": delta_pp}).fillna(0))

bar_delta(delta_pp.index, delta_pp.values,
          "Δ (pp) — Pública vs Privada (2019+ − 2018)",
          "Δ em pontos percentuais (pp)", "Categoria", horizontal=True, figsize=(7,3))

pct_amb_2018 = pct_series(df_2018, "AmbitoAdministrativo")
pct_amb_2019 = pct_series(df_2019plus, "AmbitoAdministrativo")
delta_amb = delta_pct(pct_amb_2019, pct_amb_2018)

display(pd.DataFrame({"% 2018": pct_amb_2018, "% 2019+": pct_amb_2019, "Δ (pp)": delta_amb}).fillna(0))

bar_delta(delta_amb.index, delta_amb.values,
          "Δ (pp) — Âmbito administrativo (2019+ − 2018)",
          "Δ em pontos percentuais (pp)", "Âmbito", horizontal=True, figsize=(8,3))


## 8) Δ (pp): Área CINE Geral (Top mudanças)

In [ ]:
cine18 = df_2018[df_2018["AREA_GERAL_CINE"] != "Não informado"]["AREA_GERAL_CINE"]
cine19 = df_2019plus[df_2019plus["AREA_GERAL_CINE"] != "Não informado"]["AREA_GERAL_CINE"]

pct_cine_2018 = (cine18.value_counts(normalize=True) * 100)
pct_cine_2019 = (cine19.value_counts(normalize=True) * 100)

delta_cine = delta_pct(pct_cine_2019, pct_cine_2018).sort_values(ascending=False)

tbl = pd.DataFrame({"% 2018": pct_cine_2018, "% 2019+": pct_cine_2019, "Δ (pp)": delta_cine}).fillna(0)
display(tbl.sort_values("Δ (pp)", ascending=False).head(15))

top = tbl.sort_values("Δ (pp)", ascending=False).head(15)["Δ (pp)"]
bar_delta(top.index, top.values,
          "Top 15 Δ (pp) — Área CINE Geral (2019+ − 2018)",
          "Δ em pontos percentuais (pp)", "Área CINE (Geral)",
          horizontal=True, figsize=(10,6))


## 9) Δ (dias): Tempo médio de tramitação — Geral e Top UFs (por volume)

In [ ]:
m18 = df_2018["TEMPO_TRAMITACAO_DIAS"].dropna().mean()
m19 = df_2019plus["TEMPO_TRAMITACAO_DIAS"].dropna().mean()
print(f"Tempo médio (dias) 2018:   {m18:.1f}")
print(f"Tempo médio (dias) 2019+:  {m19:.1f}")
print(f"Δ (dias) (2019+ − 2018):   {(m19-m18):.1f}")

sub_all = df_all.dropna(subset=["UF", "TEMPO_TRAMITACAO_DIAS"]).copy()
top_ufs = sub_all["UF"].value_counts().head(15).index

t18 = df_2018[df_2018["UF"].isin(top_ufs)].dropna(subset=["UF","TEMPO_TRAMITACAO_DIAS"]).groupby("UF")["TEMPO_TRAMITACAO_DIAS"].mean()
t19 = df_2019plus[df_2019plus["UF"].isin(top_ufs)].dropna(subset=["UF","TEMPO_TRAMITACAO_DIAS"]).groupby("UF")["TEMPO_TRAMITACAO_DIAS"].mean()

delta_uf = (t19.reindex(top_ufs) - t18.reindex(top_ufs)).dropna().sort_values(ascending=False)

display(pd.DataFrame({"média 2018": t18, "média 2019+": t19, "Δ (dias)": (t19 - t18)}).loc[list(delta_uf.index)].round(1))

bar_delta(delta_uf.index, delta_uf.values,
          "Δ (dias) — Tempo médio por UF (Top volume) (2019+ − 2018)",
          "Δ em dias (positivo = mais lento)", "UF",
          horizontal=True, figsize=(10,6))


## 10) Δ (pp): Modalidade dentro de cada grupo de Sede EAD (Não/Sim)

In [ ]:
dfg18 = df_2018.dropna(subset=["Modalidade_norm","is_sede_ead_flag"]).copy()
dfg19 = df_2019plus.dropna(subset=["Modalidade_norm","is_sede_ead_flag"]).copy()

tab18 = pd.crosstab(dfg18["Modalidade_norm"], dfg18["is_sede_ead_flag"], normalize="columns") * 100
tab19 = pd.crosstab(dfg19["Modalidade_norm"], dfg19["is_sede_ead_flag"], normalize="columns") * 100

idx = tab18.index.union(tab19.index)
cols = tab18.columns.union(tab19.columns)

tab18 = tab18.reindex(index=idx, columns=cols).fillna(0)
tab19 = tab19.reindex(index=idx, columns=cols).fillna(0)

delta_tab = tab19 - tab18

for sede in [0, 1]:
    if sede not in delta_tab.columns:
        continue
    label_sede = "Não" if sede == 0 else "Sim"
    d = delta_tab[sede].sort_values(ascending=False).head(15)

    bar_delta(d.index, d.values,
              f"Top 15 Δ (pp) — Modalidade dentro de Sede EAD = {label_sede} (2019+ − 2018)",
              "Δ em pontos percentuais (pp)", "Modalidade",
              horizontal=True, figsize=(10,6))
